In [17]:
import os
import time
import vtk

precision = ["Low", "Medium", "High", "Extreme", "Ultimate"]

difficulty = dict()
difficulty["Low"] = [1,4,1]
difficulty["Medium"] = [1,3,4,3,1]
difficulty["High"] = [1,3,6,8,6,3,1]
difficulty["Extreme"] = [1,3,5,6,8,6,5,3,1]
difficulty["Ultimate"] = [1,3,5,6,7,8,7,6,5,3,1]
difficultyArray = [1,3,6,5,7,8,7,5,6,3,1]

precisionLevel = precision[1]

difficultyArray = difficulty[precisionLevel]

difficultyLevel = len(difficultyArray)
verticalRotAngle = 180 / float(difficultyLevel -1)
horizontalRotAngle = [float(360) / x for x in difficultyArray]

def screenshot(filename):
    # screenshot code:
    w2if = vtk.vtkWindowToImageFilter()
    w2if.SetInput(renderWindow)
    w2if.Update()
    
    writer = vtk.vtkPNGWriter()
    writer.SetFileName(filename)
    writer.SetInput(w2if.GetOutput())
    writer.Write()

In [39]:
filename = "inputs/PistonAssembly/assembly1.stl"
 
reader = vtk.vtkSTLReader()
reader.SetFileName(filename)

In [40]:
dirs = os.listdir("inputs")

print dirs

['PistonAssembly', 'Rod']


In [41]:
polydata = vtk.vtkPolyData()
polydata = reader.GetOutput()
polydata.Update()
xmin, xmax, ymin, ymax, zmin, zmax = polydata.GetBounds()

dx = (xmax + xmin) / 2
dy = (ymax + ymin) / 2
dz = (zmax + zmin) / 2

In [42]:
mapper = vtk.vtkPolyDataMapper()
if vtk.VTK_MAJOR_VERSION <= 5:
    mapper.SetInput(reader.GetOutput())
else:
    mapper.SetInputConnection(reader.GetOutputPort())

In [43]:
camera = vtk.vtkCamera();
camera.SetPosition(0,0,1);
camera.SetFocalPoint(0, 0, 0);
camera.SetParallelProjection(1)

In [44]:
# Create a rendering window and renderer
renderer = vtk.vtkRenderer()
renderer.SetActiveCamera(camera);
renderWindow  = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(800,800)

In [45]:
# create a renderwindowinteractor
interactiveRenderer = vtk.vtkRenderWindowInteractor()
interactiveRenderer.SetRenderWindow(renderWindow)

In [46]:
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetDiffuseColor(0,0,0)
actor.SetOrigin(dx,dy,dz)

In [47]:
# Assign actor to the renderer
renderer.SetBackground(1,1,1)
renderer.AddActor(actor)
renderer.ResetCamera()

renderWindow.Render()

In [48]:
foldername = "outputs"
subfolder = "PistonAssembly"
outdir = foldername + "/" + subfolder

if not os.path.isdir(outdir):
    os.makedirs(outdir) 

objName = "assembly"
outputName = outdir + "/" + objName + "_"
    
outputExt = ".png"
counter = 1

for i in range(0, difficultyLevel):
    for x in range(0, difficultyArray[i]):
        renderer.ResetCameraClippingRange()
        screenshot(outputName + str(i) + "_" + str(x) + outputExt)
        actor.RotateZ(horizontalRotAngle[i])
        counter += 1
    actor.RotateX(verticalRotAngle)

In [11]:
# enable user interface interactor
interactiveRenderer.Initialize()
interactiveRenderer.Start()